# Initial coordinate matching

This script does the initial matching of the slidescanner (.czi file) and IMC (.mcd) slide coordinate systems.

As only few (manual) points should be provided, this fixes the scale to the vendor provided scale from the image metadata (IMC 1um/pixel, IF slidescanner 0.3246..um/pixel) and only a rigid alignment is calculated.

I found that the coordinate systems were fairly reproducible also between multiple slides (ca ~200 um error), so the transformation is estimated only on two slides.

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import pathlib
import pandas as pd
import numpy as np

In [ ]:
import workflow.scripts.utils_alignment.library as lib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import skimage.io as skio
import skimage.transform as sktranf

## Config

In [ ]:
Cout = snakemake.output
Cin = snakemake.input
Cpar = snakemake.params

In [ ]:
for f in [pathlib.Path(Cout.fn_transf_ifslide_imcslide).parent]:
    f.mkdir(exist_ok=True)

A matching pair of slidescan and IMC acquisition with 3 sites/panoramas

fn_slide = pathlib.Path(Cin.fol_slide) / 'v.zanotelli_20190509_p165_036.czi'
fol_imc = pathlib.Path(Cin.fol_imc) / '20190525_p165_slide45_ac1_vz'

Manually determined, matching points:

In [ ]:
# matching points manually determined

# coordinates: in mcd slide coordinates
p_imc = np.array((np.array((29819, 13388.8)),
         np.array((17979.1, 10165)),
         np.array((42161.2, 13107))
        ))
         
# coordinates: in czi slide coordinates
p_if = np.array((np.array((-91289.4, 36818.4)), 
        np.array((-54926.2, 26638.9)),
        np.array((-129342, 36198.8))
       ))

-> These points were determined by finding matching image regions in the plots bellow (by using the interactive matplotlib backend and zooming in):

-> This was done based on some slides from the 'overexpression' part of the data. Please look at the notebook there to see the plots

Estimate the transform based on these coordinates

In [ ]:
# scale from czi metadata
scale = np.array((-0.32465132447751116,0.32465132447751116)) #(p1_imc-p2_imc)/(p1_if-p2_if)
#offset = np.mean([p1-(p2*scale) for p1, p2 in zip(p_imc, p_if)], axis=0)

In [ ]:
# Hardcode scale
transf_scale = sktranf.AffineTransform(scale=scale)

In [ ]:
# only estimate translation and rotation
transf_translate = sktranf.estimate_transform('euclidean', transf_scale(p_if), p_imc)

In [ ]:
# combine transforms
transf_ifslide_imcslide = sktranf.AffineTransform((transf_scale + transf_translate).params)

In [ ]:
# save the transform
np.savetxt(str(Cout.fn_transf_ifslide_imcslide), transf_ifslide_imcslide.params)

In [ ]:
params = np.loadtxt(Cout.fn_transf_ifslide_imcslide)
transf_ifslide_imcslide = sktranf.AffineTransform(params)

Apply the transform to demonstrate that it works

In [ ]:
import matplotlib.transforms as transforms

In [ ]:
ifimc_transform = transforms.Affine2D(params)

In [ ]:
def show_alignment(fn_slide, fol_imc, scene_id, pano_id, transform_ifimc, channel=1):
    ax =lib.plot_czi_slide(fn_slide, channel=channel,scenes=[scene_id])
    im = ax.images[0]
    trans_data = transform_ifimc + ax.transData
    im.set_transform(trans_data)
    ax2=lib.plot_imc_slide(fol_imc, panoid=pano_id, alpha=0.5, ax=ax)
    x0, x1, y0, y1 = im.get_extent()
    p1 = transform_ifimc.transform_point((x0,y0))
    p2 = transform_ifimc.transform_point((x1,y1))
    ax.set_xlim((min(p1[0], p2[0]), max(p1[0], p2[0])))
    ax.set_ylim((min(p1[1], p2[1]), max(p1[1], p2[1])))
    plt.title('After alignment')
    return ax

show_alignment(fn_slide, fol_imc, scene_id=1, pano_id=2, transform_ifimc=ifimc_transform)

In [ ]:

show_alignment( pathlib.Path(Cin.fol_slide) / 'v.zanotelli_20190509_p165_061.czi',
               pathlib.Path(Cin.fol_imc) / '20190515_p165_slide6_ac1_vz',
               scene_id=0,
               pano_id=1,
               channel=0,               
               transform_ifimc=ifimc_transform)


In [ ]:
import sys
!conda env export -p {sys.prefix} 